# 2. JSONデータをアップロードし、信号定義に従って物理値を取得する

このケースでは、以下の方法を中心に解説します。

- JSONデータを文字列としてサーバーにアップロードする
- JSONデータをキーごとに分解する信号定義を登録する
- 上記登録した信号定義を指定してデータを取得する

## シナリオ
**「1. 時系列データを取得し、CSVで保存する」**  にて取得したCSVをJSONに変換して、新たな時系列データとして登録します。その後、JSON内に含まれていた数値を取り出します。

タイムスタンプとJSON文字列のペアをintdashにアップロードした後、信号定義を行い、デシリアライズされたデータを取得します。

## 事前準備

本シナリオを実施する前に、以下を用意する必要があります。

- データアップロード用のエッジ
- **「1. 時系列データを取得し、CSVで保存する」** で作成されたCSV

### 使用データ
本シナリオでは、以下のデータをサーバー側に準備する必要があります。  

|データ項目|本シナリオで登場するデータ名|
|:---|:---|
|時系列データを登録するエッジ|sdk_edge1|
|時系列データ(CSV) (※1)|sdk_sample.csv|
|信号定義(※2)| `json_ACCX`, `json_ACCY`, `json_ACCZ`|

(※１) 「1. 時系列データを取得し、CSVで保存する」で取得したCSVを使用してください   
(※２) JSONの信号定義を登録します。登録方法については、以下の手順を確認してください

### パッケージのimportとクライアントの生成
`intdash.Client` に与える `url` は intdashサーバーの環境情報から、 `edge_token` はログイン用エッジで発行したトークンを指定してください。  
(※ `username` と `password` でのログインも可能ですが、継続して動作する場合はエッジトークンの使用を推奨します)

In [36]:
import json
import pandas as pd

import intdash
from intdash import timeutils

# Create client

client = intdash.Client(
    url = "https://example.intdash.jp",
    edge_token = "your_token",
)

## JSON文字列を作成しタイムスタンプと関連付ける
**「1. 取得した時系列データをCSVで保存する」**  で作成したCSVデータをJSON形式に変換します。 変換後のデータは、タイムスタンプとJSON文字列が格納されたTuple型で保持します。

In [12]:
df = pd.read_csv('sdk_sample.csv' , index_col=0)
df

,sp_ACCY,sp_ACCZ,sp_ACCX
time,,,
2020-07-09T08:34:11.095032000Z,0.041493,0.217996,0.048094
2020-07-09T08:34:11.115194000Z,0.123338,0.305914,0.038043
2020-07-09T08:34:11.135356000Z,0.046133,0.436974,-0.051388
2020-07-09T08:34:11.155517000Z,-0.034988,0.233272,0.102698
2020-07-09T08:34:11.175679000Z,-0.055907,-0.164569,0.047395
...,...,...,...
2020-07-09T08:34:14.361262000Z,0.506122,0.231640,-1.372483
2020-07-09T08:34:14.381424000Z,0.354834,-0.354698,-1.570656
2020-07-09T08:34:14.401586000Z,-0.216771,0.578869,-0.578583


In [34]:
# convert format : {index -> {column -> value}}
dicts = df.to_dict(orient='index')

# convert tuple 
dict_list = list(dicts.items())

In [35]:
dict_list[0]

('2020-07-09T08:34:11.095032000Z',
 {'sp_ACCY': 0.041493, 'sp_ACCZ': 0.217996, 'sp_ACCX': 0.048094})

dict型からJSON文字列に変換します。

In [70]:
dict_list = [(d[0], json.dumps(d[1]))  for d in dict_list]

In [71]:
dict_list[0]

('2020-07-09T08:34:11.095032000Z',
 '{"sp_ACCY": 0.041493, "sp_ACCZ": 0.217996, "sp_ACCX": 0.048094}')

## タイムスタンプと関連付けられたJSON文字列をサーバーに登録する
上記で作成したタイムスタンプとJSON文字列のペアを `intdash.DataPoint` に変換したあと、サーバーに登録します。

### 計測を作成する
最初に、時系列データに紐付けるための計測データを作成します。
まず、紐付け先のエッジの情報を取得します。

In [37]:
edges = client.edges.list(name='sdk_edge1')
sdk_edge1 = edges[0]

In [38]:
sdk_edge1.name

'sdk_edge1'

In [40]:
meas = client.measurements.create(
    edge_uuid= sdk_edge1.uuid, 
    basetime=timeutils.str2timestamp(dict_list[0][0])  #  The start time specifies the time of  first data.
)

In [41]:
print(meas)

uuid: 82827deb-31b0-4093-aa64-28a717fbca06
name: 
description: 
edge_uuid: bcecc99c-d255-4666-b4a7-6b492fe0986a
duration: 0 days 00:00:00
ended: False
basetime: 2020-07-09 08:34:11.095032+00:00
basetime_type: manual
processed_ratio: 0
protected: False
markers: None
created_at: 2020-07-16 08:49:15.170176+00:00
updated_at: 2020-07-16 08:49:15.170176+00:00


### 計測に時系列データを追加する
「1. JSONデータを時系列順に並べる」 にて作成したデータを、`intdash.DataPoint` に変換します。

In [72]:
dps = [
    intdash.DataPoint(
        elapsed_time=timeutils.str2timestamp(d[0]) - meas.basetime,  # Time elapsed from the start of measurement.
        data_type=intdash.DataType.string.value, #  store JSON as string.
        channel=1, # fixed at 1.
        data_payload=intdash.data.String(data_id='json_data', value=d[1]).to_payload()
    ) 
    for d in dict_list
]

In [73]:
print(dps[1])

elapsed_time: 0 days 00:00:00.020162
ndata_type: 10
data: b'\tjson_data{"sp_ACCY": 0.123338, "sp_ACCZ": 0.305914, "sp_ACCX": 0.038043}'


変換した結果を、計測のデータポイントとして登録します。

In [74]:
client.data_points.store(
    measurement_uuid = meas.uuid,  # uuid of measurement.
    data_points = dps,
)

計測が更新されているかどうか確認します。 

In [75]:
meas = client.measurements.get(
    uuid=meas.uuid
)
print(meas)

uuid: 82827deb-31b0-4093-aa64-28a717fbca06
name: 
description: 
edge_uuid: bcecc99c-d255-4666-b4a7-6b492fe0986a
duration: 0 days 00:00:03.346878
ended: True
basetime: 2020-07-09 08:34:11.095032+00:00
basetime_type: manual
processed_ratio: 1
protected: False
markers: None
created_at: 2020-07-16 08:49:15.170176+00:00
updated_at: 2020-07-16 11:00:33.031773+00:00


## JSONに紐づく変換定義を登録する

JSON文字列内の特定のキーに対応する値について、物理値への変換方法を設定します。この設定には `intdash.ConversionJSON` を使います。作成した変換定義にはラベルを付与します。詳細については、以下を参照してください。

[create-signal-json.ipynb](../0_create-signals/create-signal-json.ipynb)  

なお、本シナリオでは以下の表の紐付けで登録しています。

|JSONのキー|物理値変換後のデータに与えるラベル|
|---|---|
|sp_ACCX|json_ACCX|
|sp_ACCY|json_ACCY|
|sp_ACCZ|json_ACCZ|


## JSONに紐づく信号定義を確認する
先程登録した信号定義が登録されているかどうか、確認します。

In [56]:
signals = client.signals.list(label='json')
for s in signals:
    print(s.label,  end=', ')

json_ACCX, json_ACCY, json_ACCZ, 

## 信号定義を指定して時系列データを取得する
**「1. 取得した時系列データをCSVで保存する」**と同様に、信号定義を指定してデータを取得します。

In [76]:
dps = client.data_points.list(
    edge_name='sdk_edge1',
    start=timeutils.str2timestamp('2020-07-09 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-10 00:00:00+09:00'),
    labels=['json_ACCX', 'json_ACCY', 'json_ACCZ']
)

In [77]:
print(dps[0])

time: 2020-07-09T08:34:11.095032000Z
measurement_uuid: 82827deb-31b0-4093-aa64-28a717fbca06
data_type: 11
channel: 1
data_id: json_ACCX
data_payload: b'\tjson_ACCX\x94\x84D\xda\xc6\x9f\xa8?'


`data_payload` をペイロードから数値に変換し、 `json_ACCX` の値が取得できることを確認します。

In [79]:
intdash.data.Float.from_payload(dps[0].data_payload).value

0.048094

JSONデータをそのまま取得したい場合は、登録時に指定したdata_id（`json_data`）を指定することで取得できます。

In [59]:
dps = client.data_points.list(
    edge_name='sdk_edge1',
    start=timeutils.str2timestamp('2020-07-09 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-10 00:00:00+09:00'),
    id_queries=[intdash.IdQuery(
        data_type=intdash.DataType.string,
        data_id='json_data',
        channel=1
    )]
)

In [62]:
print(dps[0])

time: 2020-07-09T08:34:11.095032000Z
measurement_uuid: 82827deb-31b0-4093-aa64-28a717fbca06
data_type: 10
channel: 1
data_id: json_data
data_payload: b"\tjson_data{'sp_ACCY': 0.041493, 'sp_ACCZ': 0.217996, 'sp_ACCX': 0.048094}"
